In [ ]:
# Common
import os
import cv2
import numpy as np
# Data
import tarfile
from tqdm import tqdm
from glob import glob
# Data Visuaalization
import matplotlib.pyplot as plt

In [ ]:
!pip install mtcnn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 10.3 MB/s eta 0:00:00


In [ ]:
# confirm mtcnn was installed correctly
import mtcnn
# print version
print(mtcnn.__version__)

0.1.0


In [ ]:
model = mtcnn.MTCNN()

In [ ]:
import logging
logging.basicConfig(level=logging.ERROR)
def Extract_faces(img):
  main= img.copy()
  boxes = model.detect_faces(main)
  if len(boxes) >1:
    biggest = 0
    index = -1
    for box in boxes:

      x, y, width, height = box['box']
      if width > biggest:
          biggest = width
          index+=1
    boxes = boxes[index]


  try:
    print(boxes)
    x, y, width, height = boxes[0]['box']
  except:
    x, y, width, height = boxes['box']

  try:
    main = img[y:y+height,x-30:x+width+30,:]
  except:

    try:
      main = img[y:y+height,x-20:x+width+20,:]
    except:
      try:
        main = img[y:y+height,x-10:x+width+10,:]
      except:
        try:
          main = img[y:y+height,x-5:x+width+5,:]
        except:
          main = img[y:y+height,x:x+width,:]
  main = cv2.resize(main,(150,150))
  return main[:,:,::-1]


In [ ]:
def load_data(Path):
    strLabel=list()
    images=[]
    # Access the extracted images and process them
    with tarfile.open(Path, 'r:gz') as tar:
        for member in tar.getmembers():
            if member.isfile():# Assuming images are JPEG files
                # Extract image bytes from the tarfile
                image_bytes = tar.extractfile(member).read()
                # Load the image using PIL
                image_array = np.asarray(bytearray(image_bytes), dtype=np.uint8)
                image = cv2.imdecode(image_array, cv2.IMREAD_COLOR)
                image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
                # Now you can process the image as needed
                # For example, you can display it:
                classNAme = member.name[17:member.name[17:].find('/')+17]
                strLabel.append(classNAme)
                images.append(image)
    return images,strLabel



In [ ]:
X,y=load_data('lfw-deepfunneled.tgz')

In [ ]:
%%capture
for c,img in enumerate(X):
  try:
    cv2.imwrite(f'data/img_{c}.png',Extract_faces(img))
  except:
    cv2.imwrite(f'data/Need Edit img_{c}.png',img)

In [ ]:
len(os.listdir('data/'))

13233

In [ ]:
#Images Not works in MTCNN model detect_faces
care = []
for img in os.listdir('/data'):
  if img.startswith("Need"):

    care.append(img)

In [ ]:
#you have to handel all images
care

['Need Edit img_12951.png',
 'Need Edit img_11924.png',
 'Need Edit img_9662.png',
 'Need Edit img_9836.png',
 'Need Edit img_8493.png',
 'Need Edit img_8516.png',
 'Need Edit img_7409.png',
 'Need Edit img_7484.png',
 'Need Edit img_7795.png',
 'Need Edit img_7836.png',
 'Need Edit img_7993.png',
 'Need Edit img_6559.png',
 'Need Edit img_6753.png',
 'Need Edit img_7181.png',
 'Need Edit img_7221.png',
 'Need Edit img_5246.png',
 'Need Edit img_6216.png',
 'Need Edit img_3844.png',
 'Need Edit img_4043.png',
 'Need Edit img_4200.png']

In [ ]:
#Handel undetectedd faces 
cv2.imwrite('data/img_4200.png',cv2.resize(X[4200][40:210,40:170,:],(150,150))[:,:,::-1])

True

In [ ]:
file = open('/data/target.txt','w')
for i in y:
  file.write(i+'\n')
file.close()

# Zip images 

In [ ]:
import shutil
import zipfile

def zip_folder(folder_path, output_zip_path):
    """Zips the contents of a folder into a zip file.

    Args:
        folder_path: The path to the folder to be zipped.
        output_zip_path: The path for output zip file.
    """

    with zipfile.ZipFile(output_zip_path, "w") as zip_file:
        for root, dirs, files in os.walk(folder_path):
            for file in files:
                file_path = os.path.join(root, file)
                zip_file.write(file_path)

if __name__ == "__main__":
    folder_path = "/data" #Souce folder
    output_zip_path = "data/file.zip" #zip file

    zip_folder(folder_path, output_zip_path)